In [ ]:
from tqdm import tqdm
import regex as re
import os
import pandas as pd

In [2]:
#parse new logfiles
print("Started scanning...")
result_files = {"repetition_0":{"nocrash_e1_31":{}, "nocrash_e2_31":{}, "nocrash_e3_31":{}},
                 "repetition_1":{"nocrash_e1_31":{}, "nocrash_e2_31":{}, "nocrash_e3_31":{}},
                   "repetition_2":{"nocrash_e1_31":{}, "nocrash_e2_31":{}, "nocrash_e3_31":{}}}
for root, dirs, files in os.walk("/home/maximilian/Master/eval_included_in_thesis/id20_all_levels/"):
    for dir in dirs:
        eval_reps = os.path.join(root, dir)
        if re.findall("nocrash_31", dir):
            for rep_root, rep_dirs, rep_files in os.walk(eval_reps):
                for rep_dir in rep_dirs:
                    file_lst=[]
                    rep_path = os.path.join(rep_root, rep_dir)
                    if rep_dir == "results":
                        for filename in tqdm(os.listdir(rep_path)):
                            file_lst.append(os.path.join(rep_path, filename))
                        training_rep=os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname(rep_root))))
                        eval_rep=os.path.basename(rep_root)
                        baseline=os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(rep_root))))))
                        experiment=os.path.basename(os.path.dirname(os.path.dirname(os.path.dirname(os.path.dirname(rep_root)))))
                        result_files[training_rep][eval_rep].update({f"{baseline}_{experiment}":file_lst})

Started scanning...


100%|██████████| 26/26 [00:00<00:00, 356378.77it/s]


In [4]:
weathers = {"train": [1, 6, 10, 14], "test": [3,8]}
def weather_mapping(value):
    for key, value_lst in weathers.items():
        if value in value_lst:
            return key

In [5]:
def get_ablations_dict():
    return {"bev":0, "detectboxes": 0,"speed":0, "prevnum":0, "framehandling": "unrolling", "datarep":1, 
            "augment": 0, "freeze": 0, "backbone": "resnet", "pretrained": 0, "subsampling": 0, 
            "velocity_brake_prediction": 1, "ego_velocity_prediction": 0, "init": 0,
              "augment": 0, "predict_vectors": 0, "tf_pp_rep": 0}

In [6]:
#new log files
df_lst=[]
for training_rep, eval_dict in tqdm(result_files.items()):
    for baseline, baseline_dict in eval_dict.items():
        for eval_rep, path_lst in baseline_dict.items():
            for path in path_lst:
                eval_results = pd.read_csv(path)
                df_lst.append(eval_results)
df = pd.concat(df_lst, ignore_index=True)
df["success"] = (df["timeout_blocked"] == 0) & (df["collision"] == 0)
df["success"] = df["success"].astype("int")
df["weather"] = df["weather"].map(lambda x: weather_mapping(x))
ablations_dict=get_ablations_dict()

100%|██████████| 3/3 [00:00<00:00,  6.25it/s]


In [33]:
#watch out when considering multiple training reps
groups = df[(df["backbone"]=="resnet") &(df["town"]=="Town02")].groupby(["baseline", "town","traffic", "weather", "training_rep",*ablations_dict]).agg(
    timeouts_percentage=("timeout_blocked", "mean"),
    collisions_percentage=("collision", "mean"),
    success_percentage=("success", "mean"),
)

In [34]:
(groups*100).loc[(groups*100).idxmax()]

timeouts_percentage  \
baseline town   traffic weather training_rep bev detectboxes speed prevnum framehandling datarep augment freeze backbone pretrained subsampling velocity_brake_prediction ego_velocity_prediction init predict_vectors tf_pp_rep                        
bcso     Town02 2       train   0            1   1           0     0       unrolling     3       1       0      resnet   1          0           1                         0                       0    0               0                          0.0   
                                                                                                                                                                                                                       0                          0.0   
                                1            1   1           0     0       unrolling     3       1       0      resnet   1          0           1                         0                       0    0               0                          0.0   

                                                                                                                                                                                                                                  collisions_percentage  \
baseline town   traffic weather training_rep bev detectboxes speed prevnum framehandling datarep augment freeze backbone pretrained subsampling velocity_brake_prediction ego_velocity_prediction init predict_vectors tf_pp_rep                          
bcso     Town02 2       train   0            1   1           0     0       unrolling     3       1       0      resnet   1          0           1                         0                       0    0               0                       5.333333   
                                                                                                                                                                                                                       0                       5.333333   
                                1            1   1           0     0       unrolling     3       1       0      resnet   1          0           1                         0                       0    0               0                       0.000000   

                                                                                                                                                                                                                                  success_percentage  
baseline town   traffic weather training_rep bev detectboxes speed prevnum framehandling datarep augment freeze backbone pretrained subsampling velocity_brake_prediction ego_velocity_prediction init predict_vectors tf_pp_rep                      
bcso     Town02 2       train   0            1   1           0     0       unrolling     3       1       0      resnet   1          0           1                         0                       0    0               0                   94.666667  
                                                                                                                                                                                                                       0                   94.666667  
                                1            1   1           0     0       unrolling     3       1       0      resnet   1          0           1                         0                       0    0               0                  100.000000

In [7]:
#watch out when considering multiple training reps
groups = df.groupby(["baseline", "town","traffic", "weather", "eval_rep", "training_rep",*ablations_dict]).agg(
    timeouts_percentage=("timeout_blocked", "mean"),
    collisions_percentage=("collision", "mean"),
    success_percentage=("success", "mean"),
)
groups = groups.groupby(["baseline", "town", "traffic", "weather",*ablations_dict]).agg(
    timeout_mean=("timeouts_percentage", "mean"),
    timeout_std=("timeouts_percentage", "std"),
    collisions_mean=("collisions_percentage", "mean"),
    collisions_std=("collisions_percentage", "std"),
    success_mean=("success_percentage", "mean"),
    success_std=("success_percentage", "std"),
)

In [8]:
groups=(groups*100).round(2)

In [9]:
groups

timeout_mean  \
baseline town   traffic weather bev detectboxes speed prevnum framehandling datarep augment freeze backbone pretrained subsampling velocity_brake_prediction ego_velocity_prediction init predict_vectors tf_pp_rep                 
bcso     Town01 0       train   1   1           0     0       unrolling     3       0       0      resnet   1          0           1                         0                       0    0               1                 12.44   
                1       train   1   1           0     0       unrolling     3       0       0      resnet   1          0           1                         0                       0    0               1                  4.89   
                2       train   1   1           0     0       unrolling     3       0       0      resnet   1          0           1                         0                       0    0               1                  3.11   
         Town02 0       train   1   1           0     0       unrolling     3       0       0      resnet   1          0           1                         0                       0    0               1                 16.44   
                1       train   1   1           0     0       unrolling     3       0       0      resnet   1          0           1                         0                       0    0               1                 10.67   
                2       train   1   1           0     0       unrolling     3       0       0      resnet   1          0           1                         0                       0    0               1                  7.56   

                                                                                                                                                                                                                     timeout_std  \
baseline town   traffic weather bev detectboxes speed prevnum framehandling datarep augment freeze backbone pretrained subsampling velocity_brake_prediction ego_velocity_prediction init predict_vectors tf_pp_rep                
bcso     Town01 0       train   1   1           0     0       unrolling     3       0       0      resnet   1          0           1                         0                       0    0               1                 5.81   
                1       train   1   1           0     0       unrolling     3       0       0      resnet   1          0           1                         0                       0    0               1                 5.58   
                2       train   1   1           0     0       unrolling     3       0       0      resnet   1          0           1                         0                       0    0               1                 4.37   
         Town02 0       train   1   1           0     0       unrolling     3       0       0      resnet   1          0           1                         0                       0    0               1                22.84   
                1       train   1   1           0     0       unrolling     3       0       0      resnet   1          0           1                         0                       0    0               1                16.12   
                2       train   1   1           0     0       unrolling     3       0       0      resnet   1          0           1                         0                       0    0               1                10.85   

                                                                                                                                                                                                                     collisions_mean  \
baseline town   traffic weather bev detectboxes speed prevnum framehandling datarep augment freeze backbone pretrained subsampling velocity_brake_prediction ego_velocity_prediction init predict_vectors tf_pp_rep                    
bcso     Town01 0       train   1   1           0     0       unrolling     3       0      